# Chess Position Scorer (GPU-Accelerated with Lc0)

This notebook scores chess positions using Leela Chess Zero (Lc0) with GPU acceleration.

**Requirements:**
- Google Colab with GPU runtime (A100 recommended)
- Position files in FEN format

**Output:**
- Centipawn evaluation
- Best move
- Win/Draw/Loss percentages

## 1. Setup Runtime

**IMPORTANT:** Before running, go to `Runtime > Change runtime type > GPU (A100 if available)`

In [ ]:
# Check GPU availability
!nvidia-smi

## 2. Install Lc0 and Dependencies

In [ ]:
# Install system dependencies for building Lc0
!apt-get update -qq
!apt-get install -qq -y git ninja-build clang libopenblas-dev

# Install Python dependencies
!pip install -q meson chess

print("Dependencies installed!")

In [ ]:
# Build Lc0 from source with CUDA support
# This takes ~5-10 minutes

import os

print("Cloning Lc0 repository...")
!git clone --depth 1 https://github.com/LeelaChessZero/lc0.git

print("\nBuilding Lc0 with CUDA backend (this takes ~5-10 minutes)...")
%cd lc0
!CC=clang CXX=clang++ ./build.sh

# Find the built binary
LC0_PATH = os.path.abspath("build/release/lc0")
%cd ..

print(f"\nLc0 built successfully!")
print(f"Binary location: {LC0_PATH}")

# Verify it works
!{LC0_PATH} --help | head -3

In [ ]:
# Download neural network weights
import urllib.request
import gzip
import shutil
import os

# BT4 network - latest strong network
# See https://lczero.org/dev/wiki/networks/ for alternatives
NETWORK_URL = "https://storage.lczero.org/files/networks-contrib/t1-512x15x8h-distilled-swa-3395000.pb.gz"
NETWORK_FILE = "network.pb"

print("Downloading neural network weights (~40MB)...")
urllib.request.urlretrieve(NETWORK_URL, "network.pb.gz")

print("Extracting...")
with gzip.open("network.pb.gz", "rb") as f_in:
    with open(NETWORK_FILE, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

NETWORK_PATH = os.path.abspath(NETWORK_FILE)
print(f"Network weights saved to: {NETWORK_PATH}")
print(f"Network size: {os.path.getsize(NETWORK_PATH) / 1024 / 1024:.1f} MB")

# Also set LC0_PATH in case previous cell was run in different session
LC0_PATH = os.path.abspath("lc0/build/release/lc0")

In [ ]:
# Verify Lc0 works with GPU
import os
LC0_PATH = os.path.abspath("lc0/build/release/lc0")

print("Testing Lc0...")
!{LC0_PATH} --help | head -5

print("\nChecking available backends...")
!{LC0_PATH} --show-hidden --help 2>&1 | grep -i "backend" | head -5

print("\nLc0 is ready!")

## 3. Upload Position Files

**Option A:** Upload directly from your computer

**Option B:** Mount Google Drive

In [ ]:
# OPTION A: Upload files directly
# Uncomment and run this cell to upload position files

# from google.colab import files
# import os
# 
# os.makedirs("positions", exist_ok=True)
# print("Select your position_*.txt files to upload:")
# uploaded = files.upload()
# 
# for filename in uploaded.keys():
#     os.rename(filename, f"positions/{filename}")
# 
# print(f"\nUploaded {len(uploaded)} files to 'positions/' directory")

In [ ]:
# OPTION B: Mount Google Drive
# Upload your positions folder to Google Drive first, then run this

from google.colab import drive
drive.mount("/content/drive")

# Set this to your positions folder path in Google Drive
POSITIONS_DIR = "/content/drive/MyDrive/chess_positions"

# Or create a folder and upload there
import os
os.makedirs(POSITIONS_DIR, exist_ok=True)
print(f"Position files should be in: {POSITIONS_DIR}")

In [ ]:
# Set your positions directory here
# Change this path to match where your files are

POSITIONS_DIR = "/content/drive/MyDrive/chess_positions"  # Google Drive
# POSITIONS_DIR = "/content/positions"  # Direct upload

import glob
position_files = sorted(glob.glob(f"{POSITIONS_DIR}/position_*.txt"))
position_files = [f for f in position_files if "_score" not in f]

print(f"Found {len(position_files)} position files")
if position_files:
    print(f"First file: {position_files[0]}")
    print(f"Last file: {position_files[-1]}")

## 4. Scoring Engine

In [ ]:
import subprocess
import math
import time
import sys

class Lc0Engine:
    """Wrapper for Lc0 engine with UCI protocol."""
    
    def __init__(self, lc0_path, network_path, nodes=800):
        """
        Initialize Lc0 engine.
        
        Args:
            lc0_path: Path to lc0 binary
            network_path: Path to neural network weights
            nodes: Number of nodes to search (more = stronger but slower)
        """
        self.nodes = nodes
        self.process = subprocess.Popen(
            [lc0_path, f"--weights={network_path}", "--verbose-move-stats"],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            bufsize=1
        )
        self._send("uci")
        self._wait_for("uciok")
        self._send("isready")
        self._wait_for("readyok")
    
    def _send(self, cmd):
        """Send command to engine."""
        self.process.stdin.write(cmd + "\n")
        self.process.stdin.flush()
    
    def _wait_for(self, expected):
        """Wait for expected response."""
        while True:
            line = self.process.stdout.readline().strip()
            if expected in line:
                return line
    
    def _read_until_bestmove(self):
        """Read output until bestmove, collecting info."""
        info_lines = []
        while True:
            line = self.process.stdout.readline().strip()
            if line.startswith("info"):
                info_lines.append(line)
            elif line.startswith("bestmove"):
                return info_lines, line
    
    def analyze(self, fen):
        """
        Analyze a position.
        
        Returns:
            dict with: wdl (win/draw/loss %), cp (centipawns), best_move
        """
        self._send(f"position fen {fen}")
        self._send(f"go nodes {self.nodes}")
        
        info_lines, bestmove_line = self._read_until_bestmove()
        
        # Parse best move
        best_move = bestmove_line.split()[1] if len(bestmove_line.split()) > 1 else None
        
        # Parse WDL from last info line with wdl
        wdl = None
        cp = None
        
        for line in reversed(info_lines):
            if "wdl" in line:
                parts = line.split()
                try:
                    wdl_idx = parts.index("wdl")
                    w = int(parts[wdl_idx + 1])
                    d = int(parts[wdl_idx + 2])
                    l = int(parts[wdl_idx + 3])
                    wdl = (w / 1000, d / 1000, l / 1000)  # Convert to percentages
                except (ValueError, IndexError):
                    pass
            
            if "cp" in line and cp is None:
                parts = line.split()
                try:
                    cp_idx = parts.index("cp")
                    cp = int(parts[cp_idx + 1])
                except (ValueError, IndexError):
                    pass
            
            if wdl and cp is not None:
                break
        
        # Convert WDL to centipawns if cp not directly available
        if cp is None and wdl:
            cp = self.wdl_to_cp(wdl)
        
        return {
            "wdl": wdl,
            "cp": cp,
            "best_move": best_move,
            "nodes": self.nodes
        }
    
    @staticmethod
    def wdl_to_cp(wdl):
        """
        Convert WDL to centipawns.
        
        Uses the formula: cp = 111.714 * tan(1.5620688 * (win_rate - 0.5))
        where win_rate = win + draw/2
        """
        if wdl is None:
            return None
        
        w, d, l = wdl
        win_rate = w + d / 2
        
        # Clamp to avoid tan() blowing up
        win_rate = max(0.001, min(0.999, win_rate))
        
        cp = 111.714 * math.tan(1.5620688 * (win_rate - 0.5))
        return int(round(cp))
    
    def close(self):
        """Shutdown engine."""
        try:
            self._send("quit")
            self.process.wait(timeout=5)
        except:
            self.process.kill()


def format_cp(cp):
    """Format centipawns as readable string."""
    if cp is None:
        return "?"
    pawns = cp / 100
    return f"{pawns:+.2f}"


print("Scoring engine defined!")

## 5. Score Positions

In [ ]:
# Configuration
NODES = 800  # Nodes to search per position (higher = stronger but slower)
             # 400 = fast (~0.5s), 800 = balanced (~1s), 1600 = strong (~2s), 10000 = very strong (~10s)

FORCE_RESCORE = False  # Set to True to re-score positions that already have scores

print(f"Configuration:")
print(f"  Nodes per position: {NODES}")
print(f"  Force re-score: {FORCE_RESCORE}")
print(f"  Positions directory: {POSITIONS_DIR}")

In [ ]:
import os
import time
from pathlib import Path

def format_time(seconds):
    """Format seconds as human-readable string."""
    if seconds < 60:
        return f"{seconds:.0f}s"
    elif seconds < 3600:
        mins = seconds // 60
        secs = seconds % 60
        return f"{mins:.0f}m {secs:.0f}s"
    else:
        hours = seconds // 3600
        mins = (seconds % 3600) // 60
        return f"{hours:.0f}h {mins:.0f}m"

# Find positions to score
position_files = sorted(glob.glob(f"{POSITIONS_DIR}/position_*.txt"))
position_files = [f for f in position_files if "_score" not in f]

if not FORCE_RESCORE:
    to_score = []
    for pf in position_files:
        score_file = pf.replace(".txt", "_score.txt")
        if not os.path.exists(score_file):
            to_score.append(pf)
    already_scored = len(position_files) - len(to_score)
else:
    to_score = position_files
    already_scored = 0

print(f"Total position files: {len(position_files)}")
print(f"Already scored: {already_scored}")
print(f"To score: {len(to_score)}")

if not to_score:
    print("\nAll positions already scored! Set FORCE_RESCORE=True to re-score.")

In [ ]:
# Run scoring
if to_score:
    print(f"\nInitializing Lc0 engine...")
    engine = Lc0Engine(LC0_PATH, NETWORK_PATH, nodes=NODES)
    print("Engine ready!\n")
    
    scored = 0
    errors = 0
    start_time = time.time()
    
    try:
        for i, pos_file in enumerate(to_score, 1):
            try:
                # Read FEN
                with open(pos_file, "r") as f:
                    fen = f.read().strip()
                
                # Analyze
                result = engine.analyze(fen)
                
                # Write score file
                score_file = pos_file.replace(".txt", "_score.txt")
                cp_str = format_cp(result["cp"])
                
                # Format: score, best_move, nodes, wdl
                wdl_str = f"{result['wdl'][0]:.1%}/{result['wdl'][1]:.1%}/{result['wdl'][2]:.1%}" if result['wdl'] else "?"
                score_content = f"{cp_str}\n{result['best_move']}\n{NODES}\n{wdl_str}"
                
                with open(score_file, "w") as f:
                    f.write(score_content)
                
                scored += 1
                
                # Progress update
                elapsed = time.time() - start_time
                rate = scored / elapsed if elapsed > 0 else 0
                eta = (len(to_score) - i) / rate if rate > 0 else 0
                
                progress = i / len(to_score) * 100
                print(f"\r[{progress:5.1f}%] {i}/{len(to_score)} | {format_time(elapsed)} elapsed | ETA: {format_time(eta)} | {rate:.1f} pos/s | Last: {cp_str}", end="")
                
            except Exception as e:
                errors += 1
                print(f"\nError on {pos_file}: {e}")
    
    except KeyboardInterrupt:
        print("\n\nInterrupted by user.")
    
    finally:
        engine.close()
    
    # Final stats
    elapsed = time.time() - start_time
    print(f"\n\n{'='*50}")
    print(f"Scoring complete!")
    print(f"  Positions scored: {scored}")
    print(f"  Errors: {errors}")
    print(f"  Total time: {format_time(elapsed)}")
    print(f"  Average speed: {scored/elapsed:.2f} positions/second")
    print(f"  Output: {POSITIONS_DIR}")

## 6. Download Results

In [ ]:
# Check scored files
score_files = sorted(glob.glob(f"{POSITIONS_DIR}/position_*_score.txt"))
print(f"Total score files: {len(score_files)}")

# Show sample
if score_files:
    print(f"\nSample score file ({score_files[0]}):")
    with open(score_files[0], "r") as f:
        print(f.read())

In [ ]:
# OPTION: Download as ZIP (for direct upload method)
# Uncomment to create and download a ZIP of all scored positions

# import shutil
# from google.colab import files
# 
# shutil.make_archive("scored_positions", "zip", POSITIONS_DIR)
# files.download("scored_positions.zip")
# print("Download started!")

## Score File Format

Each `position_XXXXXX_score.txt` contains:
```
+0.45           # Centipawn evaluation (+ = white better)
e2e4            # Best move
800             # Nodes searched
52.3%/35.2%/12.5%  # Win/Draw/Loss percentages
```